#Ejercicio 17



In [ ]:
import nltk
nltk.download("popular")
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.stats import entropy
from scipy.special import kl_div
from sklearn.neighbors import NearestNeighbors

pd.options.display.float_format = '{:20,.5f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1l5xbJPYbcoAxpSKPxVdkQU9on6Pv2X6k'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('contents_text_sample.csv')

In [ ]:
texts = pd.read_csv('contents_text_sample.csv')
texts.head()

,id,title,text
0,50,Anacardiaceae,{{Ficha de taxón\n| name = Anacardiáceas\n| im...
1,62,Antipartícula,[[Archivo:Materia antimateria.png|thumb|200px|...
2,78,Año bisiesto,'''Año bisiesto''' es una expresión que deriva...
3,127,Avena,{{otros usos|Avena sativa|la avena común}}\n{{...
4,160,Alfabeto,{{distinguir|analfabeto|alfabetización}} \n[[A...


In [ ]:
texts = texts.dropna().reset_index(drop=True)
texts

,id,title,text
0,50,Anacardiaceae,{{Ficha de taxón\n| name = Anacardiáceas\n| im...
1,62,Antipartícula,[[Archivo:Materia antimateria.png|thumb|200px|...
2,78,Año bisiesto,'''Año bisiesto''' es una expresión que deriva...
3,127,Avena,{{otros usos|Avena sativa|la avena común}}\n{{...
4,160,Alfabeto,{{distinguir|analfabeto|alfabetización}} \n[[A...
...,...,...,...
206595,10010394,Nicoleta-Ancuţa Bodnar,#REDIRECCIÓN [[Nicoleta-Ancuța Bodnar]]
206596,10010425,Sebastian Szczepański,{{Ficha de deportista\n|nombre = ...
206597,10010518,Manu (Tripura),{{Ficha de entidad subnacional\n|nombre comple...
206598,10010570,ISRU,#REDIRECCIÓN [[Utilización de recursos in situ]]


In [ ]:
tamaños = texts.apply(lambda x: len(x.text), axis = 1).to_list()
tamaños

[5261,
 13873,
 11639,
 11747,
 22535,
 1749,
 4463,
 450,
 20439,
 34934,
 25,
 3846,
 21183,
 133434,
 110402,
 4851,
 203114,
 76902,
 19968,
 861,
 3951,
 4439,
 34401,
 14413,
 13468,
 2277,
 2395,
 6363,
 189851,
 31206,
 22621,
 23,
 54033,
 32240,
 9038,
 3319,
 455,
 2572,
 6499,
 52615,
 5461,
 8898,
 291032,
 46159,
 46267,
 27,
 2404,
 1630,
 11641,
 34,
 4949,
 2994,
 27027,
 17495,
 72722,
 4068,
 10250,
 37327,
 8453,
 130,
 22717,
 17,
 14358,
 42802,
 27,
 64710,
 13466,
 49812,
 5872,
 2215,
 45453,
 11781,
 62749,
 4977,
 41,
 18,
 3264,
 11247,
 6992,
 6681,
 20,
 28765,
 8291,
 5992,
 11085,
 525,
 93475,
 9245,
 2299,
 14035,
 154122,
 187372,
 25,
 598,
 25579,
 4629,
 8130,
 24941,
 33638,
 32,
 4649,
 28,
 52,
 6171,
 36507,
 27716,
 28051,
 31,
 32662,
 1654,
 10357,
 28,
 28,
 8689,
 20439,
 3801,
 32,
 6965,
 24531,
 30,
 178613,
 27203,
 34971,
 6740,
 36,
 5389,
 7779,
 37555,
 69595,
 32097,
 52168,
 14379,
 40985,
 59240,
 8168,
 23967,
 325852,
 31155,


In [ ]:
np.percentile(tamaños, 99)

39620.510000000475

In [ ]:
np.percentile(tamaños, 1)

21.0

In [ ]:
texts = texts.loc[(texts['text'].str.len() < 40000) & (texts['text'].str.len() > 21)].reset_index(drop=True)
texts

,id,title,text
0,50,Anacardiaceae,{{Ficha de taxón\n| name = Anacardiáceas\n| im...
1,62,Antipartícula,[[Archivo:Materia antimateria.png|thumb|200px|...
2,78,Año bisiesto,'''Año bisiesto''' es una expresión que deriva...
3,127,Avena,{{otros usos|Avena sativa|la avena común}}\n{{...
4,160,Alfabeto,{{distinguir|analfabeto|alfabetización}} \n[[A...
...,...,...,...
201857,10010394,Nicoleta-Ancuţa Bodnar,#REDIRECCIÓN [[Nicoleta-Ancuța Bodnar]]
201858,10010425,Sebastian Szczepański,{{Ficha de deportista\n|nombre = ...
201859,10010518,Manu (Tripura),{{Ficha de entidad subnacional\n|nombre comple...
201860,10010570,ISRU,#REDIRECCIÓN [[Utilización de recursos in situ]]


In [ ]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words=nltk.corpus.stopwords.words('spanish'), max_features=100000)

In [ ]:
matrix = vectorizer.fit_transform(texts['text'])

In [ ]:
buscador = NearestNeighbors(algorithm='brute', metric='cosine')
buscador.fit(matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
query = "retablo iglesia"

In [ ]:
vector = vectorizer.transform([query])

In [ ]:
dist, indexes = buscador.kneighbors(vector, n_neighbors=4)

In [ ]:
for d,i in zip(dist.flatten(), indexes.flatten()):
  print("A distancia", d,"el ID:", i, "el texto", texts['text'][i])

A distancia 0.18650515872724382 el ID: 4589 el texto #REDIRECCIÓN [[Retablo]]
A distancia 0.34258088682372945 el ID: 189137 el texto #REDIRECCIÓN [[Retablo mayor de la iglesia de San Millán de los Balbases]]
A distancia 0.44940011919566436 el ID: 79625 el texto #REDIRECCIÓN [[Museo del Retablo (Burgos)]]
A distancia 0.45648830023481923 el ID: 129389 el texto #REDIRECCIÓN [[Retablo de Bernabé]]


In [ ]:
texts.loc[indexes.flatten()[1]]

id                                                 9357972
title    Retablo mayor de la iglesia de san millan de l...
text     #REDIRECCIÓN [[Retablo mayor de la iglesia de ...
Name: 189137, dtype: object